In [5]:
#!pip install xgboost
import os
import sys
import warnings

PROJ_PATH = os.path.abspath(os.path.join(os.getcwd(), ".."))

sys.path.append(PROJ_PATH) 

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.metrics import mean_squared_error
import itertools
from pandas.api.types import CategoricalDtype

from datetime import date 
from datetime import datetime, timedelta 
import datetime as DT 
import time

import xgboost as xgb 

import helper_functions as hfs 
from scipy.stats import linregress 
import math
from sklearn.decomposition import PCA

from sklearn import metrics
from sklearn.metrics import confusion_matrix

warnings.simplefilter("ignore", UserWarning)
warnings.filterwarnings("ignore")

pd.set_option("display.max_columns", None)  # to see all columns of dataframe
pd.set_option("display.max_rows", None)  # to see all rows of dataframe
pd.set_option("display.max_colwidth", None)  # to see the full (un-truncated) dataframe field/cell value 

In [6]:
import configparser

env_config = configparser.ConfigParser()
env_config.read(f'{PROJ_PATH}/kye/env_config.ini')

# <<<<<<<<<<<<<<< >>>>>>>>>>>>>>>>>>>>>>> #

SPLIT_DATE = pd.to_datetime(env_config['ENV_VARS']['SPLIT_DATE'])
ML_DATA_FOLDER_PATH = env_config['LOCAL_PATHS']['ML_DATA_FOLDER_PATH']
PROCESSING_TEMP_FOLDER_PATH = env_config['LOCAL_PATHS']['PROCESSING_TEMP_FOLDER_PATH'] 

OUTPUTS_FOLDER_PATH = env_config['LOCAL_PATHS']['OUTPUT_FOLDER_PATH'] + 'D1/'

#@@@@@@ **********************<<<<<<<<<<<<#

print("SPLIT_DATE: ", SPLIT_DATE)

TODAY = str(date.today())
FROM_DATE = pd.to_datetime(date.today(), format='%Y-%m-%d') - DT.timedelta(days=((365*4) + 60)) # "2016-01-01"
TO_DATE = str(date.today())
TWO_YEARS_BEFORE_FROM_DATE = pd.to_datetime(FROM_DATE, format='%Y-%m-%d') - DT.timedelta(days=730)
FROM_DATE 

SPLIT_DATE:  2023-08-09 00:00:00


Timestamp('2019-06-11 00:00:00')

In [7]:
analytics_rec_dict = {}
model_feed_dict = {}


In [22]:
def get_last_N_days(N_days):
    unique_dates_list = [x for x in analytics_df['Date'].unique()]
    split_date_index = unique_dates_list.index(SPLIT_DATE)
    N_days_dates = unique_dates_list[split_date_index-(N_days-1):split_date_index+1]
    return N_days_dates 

In [23]:
files_in_ML_DATA = os.listdir(f"{PROJ_PATH}/{ML_DATA_FOLDER_PATH}") 
files_in_ML_DATA


['all_stocks_ohlcv.pkl',
 'stocks_current_fundamentals_1340_2023-08-09.pkl',
 'indices_daily_ohlcv_2023-08-09.pkl',
 '.ipynb_checkpoints']

In [24]:
files_in_PROCESSING_TEMP = os.listdir(f"{PROJ_PATH}/{PROCESSING_TEMP_FOLDER_PATH}") 
files_in_PROCESSING_TEMP


['RIPOs_df.pkl',
 'analytics_feature_ind.pkl',
 'indices_daily_ohlcv_2023-08-09.pkl',
 '.ipynb_checkpoints',
 'OHLCV_all_stocks_ohlcv_data_prep.pkl',
 'predicted_EPs.pkl']

In [25]:
# todays_recs = pd.read_pickle("./Dashboard/todays_rec.pkl")
# print(todays_recs.shape)
# todays_recs.head()

In [27]:
analytics_df = pd.read_pickle( f"{PROJ_PATH}/{PROCESSING_TEMP_FOLDER_PATH}/{[x for x in files_in_PROCESSING_TEMP if 'analytics_feature_ind' in x][0]}") 
analytics_df['Vol_price'] = analytics_df['OHLC_price'] * analytics_df['Adj_Vol'] 

print("#Tickers: ", analytics_df['Ticker'].nunique())
print(analytics_df.shape)
analytics_df.tail(2) 

#Tickers:  1340
(334383, 310)


,company_Name,Ticker,Date,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Vol,Exchange,OHLC_price,Adj_Close_before_1d,Adj_Close_before_2d,Adj_Close_before_3d,Adj_Close_before_5d,Adj_Close_before_10d,Adj_Close_before_20d,Adj_Close_before_120d,Adj_Close_before_240d,Adj_Close_before_300d,Adj_Close_change_per_1d,Adj_Close_change_per_2d,Adj_Close_change_per_3d,Adj_Close_change_per_5d,Adj_Close_change_per_10d,Adj_Close_change_per_20d,Adj_Close_change_per_120d,Adj_Close_change_per_240d,Adj_Close_change_per_300d,Adj_Close_change_per_1d_mean10,Cam_Range_1_days,Cam_PP_day_1,Cam_R3_day_1,Cam_S3_day_1,Cam_Range_1P_days,Cam_PP_day_1P,Cam_R3_day_1P,Cam_S3_day_1P,Cam_PP_day_1_per,Cam_R3_day_1_per,Cam_S3_day_1_per,Cam_PP_day_1P_per,Cam_R3_day_1P_per,Cam_S3_day_1P_per,Cam_Range_5_days,Cam_PP_day_5,Cam_R1_day_5,Cam_S1_day_5,Cam_Range_5P_days,Cam_PP_day_5P,Cam_R1_day_5P,Cam_S1_day_5P,Cam_PP_day_5_per,Cam_R1_day_5_per,Cam_S1_day_5_per,Cam_PP_day_5P_per,Cam_R1_day_5P_per,Cam_S1_day_5P_per,Adj_High_2_days_per,Adj_Low_2_days_per,Cam_PP_day_2_per,Cam_R1_day_2_per,Cam_S1_day_2_per,Adj_High_20_days_per,Adj_Low_20_days_per,Cam_PP_day_20_per,Cam_R1_day_20_per,Cam_S1_day_20_per,Adj_High_120_days_per,Adj_Low_120_days_per,Cam_PP_day_120_per,Cam_R1_day_120_per,Cam_S1_day_120_per,Adj_High_240_days_per,Adj_Low_240_days_per,Cam_PP_day_240_per,Cam_R1_day_240_per,Cam_S1_day_240_per,RSI_9,RSI_14,RSI_20,volatility_126,max_drop_126,volatility_20,max_drop_20,TR,ATR_3,volatility_5,volatility_3,volatility_2,max_drop_5,max_drop_3,ADR_percent_5d,ADR_percent_20d,BB_2_up_20_Adj_Close,BB_2_dn_20_Adj_Close,BB_3_up_20_Adj_Close,BB_3_dn_20_Adj_Close,Adj_Close_lag1,Adj_Close_MA200,Adj_Close_MA150,Adj_Close_MA100,Adj_Close_MA90,Adj_Close_MA50,Adj_Close_MA40,Adj_Close_MA26,Adj_Close_MA20,Adj_Close_MA15,Adj_Close_MA12,Adj_Close_MA10,Adj_Close_MA5,Adj_Close_MA3,Adj_Close_MA2,D1_Close_volatility_percnt,D1_Close_volatility_percnt_before_1d,D1_Close_volatility_percnt_before_2d,D1_Close_volatility_percnt_before_3d,D1_Close_volatility_percnt_before_5d,D1_Close_volatility_percnt_before_10d,D1_Close_volatility_percnt_before_20d,price_voilatility_change_last_1d,price_voilatility_change_last_2d,price_voilatility_change_last_3d,price_voilatility_change_last_5d,price_voilatility_change_last_10d,price_voilatility_change_last_20d,gap_percent,gap_5day_max,gap_5day_min,High_52_weeks,Low_52_weeks,High_26_weeks,Low_26_weeks,High_4_weeks,Low_4_weeks,High_1_week,Low_1_week,body_percent,body_per_5d_mean,body_per_20d_mean,body_per_3month_mean,Long_MA20_crossed_MA200_in_5d,Short_MA20_crossed_MA200_in_5d,Long_MA10_crossed_MA50_in_5d,Short_MA10_crossed_MA50_in_2d,Long_MA20_crossed_MA200_in_2d,Short_MA20_crossed_MA200_in_2d,Long_MA10_crossed_MA50_in_2d,EMA_5,EMA_9,EMA_10,EMA_20,ATR,KCupper,KClower,High_Open_diff,Low_Open_diff,High_Low_diff,High_Open_diff_5d_max,High_Open_diff_5d_min,Low_Open_diff_5d_max,Low_Open_diff_5d_min,High_Low_diff_5d_max,High_Low_diff_5d_min,straight_down,straight_up,straight_down_per,straight_up_per,Close_Low_diff,Close_Open_diff,straight_up_close_per,Adj_Vol_lag1,Adj_Vol_MA60,Adj_Vol_MA20,Adj_Vol_MA10,Adj_Vol_MA5,Adj_Vol_MA2,Adj_Vol_STD20,Adj_Vol_STD10,Adj_Vol_STD5,Adj_Vol_STD2,On_Balance_Adj_Vol,Adj_Vol_before_1d,Adj_Vol_before_2d,Adj_Vol_before_3d,Adj_Vol_before_5d,Adj_Vol_before_10d,Adj_Vol_before_20d,Adj_Vol_change_per_1d,Adj_Vol_change_per_2d,Adj_Vol_change_per_3d,Adj_Vol_change_per_5d,Adj_Vol_change_per_10d,Adj_Vol_change_per_20d,VWAP_Adj_Close5,VWAP_Adj_Close3,VWAP_Adj_Close2,5_day_negatives,10_day_negatives,Adj_Vol_1d,Adj_Vol_EMA_prev_5d,Adj_Vol_EMA_prev_20d,EMA_prev_20d_DIV_Adj_Vol,EMA_prev_5d_DIV_Adj_Vol,Opening_Gap_Percent,day_Open_high_per,day_Open_low_per,day_Open_close_per,ATR_3_DIV_OHLC_price,Cam_R1_day_5P_DIV_Cam_R1_day_5,Cam_S1_day_5P_DIV_Cam_S1_day_5,Cam_R3_day_1P_DIV_Cam_R3_day_1,Cam_S3_day_1P_DIV_Cam_S3_day_1,Adj_Close_DIV_EMA_9,EMA_9_DIV_MA50,MA50_DIV_MA150,MA150_DIV_MA200,Close_DIV_High_52_weeks,Close_DIV_Low_52_weeks,Close_DIV_High_4_weeks,Close_DIV_Low_4_weeks,Clos

In [50]:
n = 5
analytics_df[f'Last_{n}_Days_Close'] = analytics_df['Adj_Close'].rolling(window=n).max() 
analytics_df[f'Last_{n}_Days_Open'] = analytics_df['Adj_Open'].rolling(window=n).min() 

analytics_df[f'High_on_day_before_{n}_Days'] = analytics_df['Adj_High'].shift(5)
analytics_df[f'Low_on_day_before_{n}_Days'] = analytics_df['Adj_Low'].shift(5) 
analytics_df[f'Vol_on_day_before_{n}_Days'] = analytics_df['Adj_Vol'].shift(5) 
analytics_df[f'Vol_on_day_before_{n+1}_Days'] = analytics_df['Adj_Vol'].shift(6) 

consolidating_5d = analytics_df[ (analytics_df['ADR_percent_20d'] > 3.2 ) & 
                                 (analytics_df['High_on_day_before_5_Days'] > 0.995 * analytics_df[f'Last_5_Days_Close'] ) &
                                 ( 1.005 * analytics_df['Low_on_day_before_5_Days'] < analytics_df[f'Last_5_Days_Open'] ) &
                                 (analytics_df['Vol_on_day_before_5_Days'] > 20000000 ) &
                                 (analytics_df['Adj_Close'] > 1 ) & 
                                 (analytics_df['Date'].isin(get_last_N_days(3))) 
                                 
                              ]['Ticker'].unique() 

consolidating_5d = [x for x in consolidating_5d ]
print(consolidating_5d) 
analytics_rec_dict['consolidating_5d'] = consolidating_5d 
len(consolidating_5d) 


['ACHR', 'ADMP', 'AMD', 'DRVN', 'FUBO', 'GDRX', 'INVZ', 'RAD', 'SNAP', 'SPR', 'TGTX', 'YELL']


12

## Extended fall/rise 

In [51]:
## Tickers that grew by 80%, 100% or 200%+ in last 1, 2 or 3 days
extended_fall = analytics_df[ ( (analytics_df['5_day_negatives'] >=4 ) |
                                       (analytics_df['10_day_negatives'] >=8 ) 
                                     ) &
                                     (analytics_df['Date'].isin(get_last_N_days(2))) &
                                     (analytics_df['Adj_Close']>=0.1) &
                                     (analytics_df['Adj_Close']<=500) &
                                     ( analytics_df['Adj_Close_change_per_3d'] < -0.10 ) &
                                     (analytics_df['Adj_Vol']>=1500000) &
                                     ( analytics_df['Adj_Vol'] >= 2 * analytics_df['Adj_Vol_EMA_prev_20d'] )
                                    ]['Ticker'].unique() 
print(extended_fall)
extended_fall = [x for x in extended_fall]
analytics_rec_dict['extended_fall_days'] = extended_fall
len(extended_fall)


['AMRS' 'PTRA' 'UPST' 'WW']


4

In [52]:
## Tickers that grew by 80%, 100% or 200%+ in last 1, 2 or 3 days
extended_rise = analytics_df[ ( (analytics_df['5_day_negatives'] <=2 ) |
                                       (analytics_df['10_day_negatives'] <=2 ) 
                                     ) &
                                     (analytics_df['Date'].isin(get_last_N_days(2))) &
                                     (analytics_df['Adj_Close']>=0.1) &
                                     (analytics_df['Adj_Close']<=500) &
                                     ( analytics_df['Adj_Close_change_per_3d'] > 0.1 ) &
                                     (analytics_df['Adj_Vol']>=1500000) &
                                     ( analytics_df['Adj_Vol'] >= 3 * analytics_df['Adj_Vol_EMA_prev_20d'] )
                                    ]['Ticker'].unique() 
print(extended_rise)
extended_rise = [x for x in extended_rise]
analytics_rec_dict['extended_rise_days'] = extended_rise
len(extended_rise)


['TDS' 'TRHC']


2

## Dipbuy plays || or Short on first red day

In [53]:
## Tickers that grew by 80%, 100% or 200%+ in last 1, 2 or 3 days
ramp_2_days = analytics_df[( (analytics_df['Adj_Close_change_per_1d']>0.5) |
                               (analytics_df['Adj_Close_change_per_2d']>0.6) | 
                               (analytics_df['Adj_Close_change_per_3d']>0.8) |
                               (analytics_df['Adj_Close_change_per_5d']>1) |
                              (analytics_df['Adj_Close_change_per_10d']>1.5) |
                              (analytics_df['Adj_Close_change_per_20d']>3) 
                             ) &
                             (analytics_df['Date'].isin(get_last_N_days(2))) &
                             (analytics_df['OHLC_price']>=1) &
                             (analytics_df['Adj_Vol']>=1500000) &
                             ( analytics_df['Adj_Vol'] >= 4 * analytics_df['Adj_Vol_EMA_prev_20d'] )
                            ]['Ticker'].unique() 
print(ramp_2_days)
ramp_2_days = [x for x in ramp_2_days]
analytics_rec_dict['ramp_2_days_for_dpb_or_frd'] = ramp_2_days
len(ramp_2_days)


['BNTC' 'TNGX']


2

## First Green Day

In [54]:
FGD_df = analytics_df[ ( #( analytics_df['Adj_Vol'] >= 5 * analytics_df['Adj_Vol_EMA_prev_5d'] ) &
                            ( analytics_df['Adj_Vol'] >= 3 * analytics_df['Adj_Vol_EMA_prev_20d'] )
                           ) &
                         (analytics_df['Adj_Vol']>=1500000) &
                         (analytics_df['Vol_Price']>=1000000) &
                         ( ( analytics_df['Adj_Close_change_per_1d'] > 0.4 ) |
                           ( analytics_df['Adj_Close'] > 1.25 * analytics_df['Adj_Open']) |
                           ( analytics_df['Adj_High'] > 1.45 * analytics_df['Adj_Open']) 
                          
                         ) &
                         #( analytics_df['Adj_Close_change_per_1d'] <= 1.5 ) &
                         #( analytics_df['Adj_Open'] > analytics_df['Adj_Close_before_1d'] ) &
                         (analytics_df['Adj_Low']<=500) &
                         #(analytics_df['Date']==SPLIT_DATE)
                        (analytics_df['Date'].isin(get_last_N_days(1))) 
                        ].reset_index(drop=True)
print(FGD_df.shape) 
FGD_list =  list(set(FGD_df['Ticker'].values))
analytics_rec_dict['FGD'] =FGD_list
FGD_list 


(2, 378)


['TNGX', 'BNTC']

In [59]:
PP_range = [x for x in analytics_df[ 
              ( analytics_df['Cam_R1_day_5P'] > 1.1 * analytics_df['Cam_R1_day_5'] ) &
              ( analytics_df['Cam_S1_day_5P'] > 1.2 * analytics_df['Cam_S1_day_5'] ) &
              ( analytics_df['Cam_R3_day_1P'] > 1.2 * analytics_df['Cam_R3_day_1'] ) &
              ( analytics_df['Cam_S3_day_1P'] > 1.2 * analytics_df['Cam_S3_day_1'] ) &
              (analytics_df['Vol_Price'] > 1000000 ) &
              (analytics_df['Adj_Vol'] >= 700000 ) &
              (analytics_df['Date'].isin(get_last_N_days(1)))
            ]['Ticker'].values]

analytics_rec_dict['PP_range'] = list(set(PP_range))
print(len(analytics_rec_dict['PP_range']))
analytics_rec_dict['PP_range'] 

15


['LPRO',
 'DRUG',
 'PALI',
 'TWOU',
 'RYAM',
 'YELL',
 'UPST',
 'GRWG',
 'VRM',
 'AMRS',
 'DOCS',
 'SDC',
 'WE',
 'RBLX',
 'NVEI']

## Cross over with good ADR

In [62]:
long_crossover = analytics_df[ ( analytics_df['ADR_percent_20d'] > 5 ) &
                                  ( ( analytics_df['Long_MA10_crossed_MA50_in_2d'] == True ) |
                                    ( analytics_df['Long_MA20_crossed_MA200_in_2d'] == True ) 
                                  )&
                                (analytics_df['Adj_Vol'] >= 5000000 ) &
                                  #(analytics_df['body_per_3month_mean'] > analytics_df['body_per_3month_mean'].mean() ) &
                                  (analytics_df['Adj_Vol_EMA_prev_20d'] > 150000 ) &
                                  (analytics_df['Date'].isin(get_last_N_days(3)))
                                ].reset_index(drop=True)['Ticker'].values

print(long_crossover)
long_crossover = [x for x in long_crossover]
analytics_rec_dict['long_crossover'] = long_crossover
len(long_crossover)

['DRUG' 'FSR' 'NKTR' 'NKTR' 'SVFD' 'TNGX']


6

In [63]:
short_crossover = analytics_df[ ( analytics_df['ADR_percent_20d'] > 5.5 ) &
                                  ( ( analytics_df['Short_MA10_crossed_MA50_in_2d'] == True ) |
                                    ( analytics_df['Short_MA20_crossed_MA200_in_2d'] == True ) 
                                  )&
                                (analytics_df['Adj_Vol'] >= 1000000 ) &
                                  #( analytics_df['TTM'] == 1 ) &
                                  #(analytics_df['body_per_3month_mean'] > analytics_df['body_per_3month_mean'].mean() ) &
                                  (analytics_df['Adj_Vol_EMA_prev_20d'] > 200000 ) &
                                  (analytics_df['Adj_Close']>= 1.5) &
                                  (analytics_df['Date'].isin(get_last_N_days(3)))
                                ].reset_index(drop=True)['Ticker'].values

print(short_crossover)
short_crossover = [x for x in short_crossover]
analytics_rec_dict['short_crossover'] = short_crossover
len(short_crossover) 

['ABCL' 'AMWL' 'AMWL' 'APLD' 'ASPN' 'BFLY' 'CYH' 'FSR' 'FTCH' 'IMGN' 'ME'
 'RUN' 'TGTX']


13

In [64]:
# Calculate the range and body of each candle
analytics_df['range'] = abs(analytics_df['Adj_Open'] - analytics_df['Adj_Close'])
analytics_df['body'] = abs(analytics_df['Adj_Open'] - analytics_df['Adj_Close'].shift(1))

# Identify bearish engulfing candles
analytics_df['bearish_engulfing'] = ((analytics_df['Adj_Close'] < analytics_df['Adj_Open']) & 
                           (analytics_df['Adj_Close'] < analytics_df['Adj_Open'].shift(1)) & 
                           (analytics_df['Adj_Open'] >= analytics_df['Adj_Close'].shift(1)) &
                           (analytics_df['body'] < analytics_df['body'].shift(1)) &
                           (analytics_df['range'] > analytics_df['range'].shift(1)))


In [65]:
bearish_engulfing = analytics_df[ ( analytics_df['ADR_percent_5d'] > 5.5 ) &
                                   (analytics_df['bearish_engulfing']==True) &
                                  #(analytics_df['Adj_Vol_EMA_prev_20d'] > 150000 ) &
                                  (analytics_df['Adj_Vol'] >= 1.5 * analytics_df['Adj_Vol_EMA_prev_20d']) &
                                  (analytics_df['Adj_Close']>= 1.5) &
                                  (analytics_df['Adj_Close']<=250) &
                                  (analytics_df['Date'].isin(get_last_N_days(1)))
                                ].reset_index(drop=True)['Ticker'].values


print(bearish_engulfing)
bearish_engulfing = [x for x in bearish_engulfing]
analytics_rec_dict['bearish_engulfing'] = bearish_engulfing
len(bearish_engulfing)

['AMWL' 'AVDL' 'BFLY' 'REI' 'RVNC' 'SERA']


6

In [66]:

analytics_df['range'] = abs(analytics_df['Adj_Open'] - analytics_df['Adj_Close'])
analytics_df['body'] = abs(analytics_df['Adj_Open'] - analytics_df['Adj_Close'].shift(1))

# Identify bullish engulfing candles
analytics_df['bullish_engulfing'] = ((analytics_df['Adj_Close'] > analytics_df['Adj_Open']) & 
                           (analytics_df['Adj_Close'] > analytics_df['Adj_Open'].shift(1)) & 
                           (analytics_df['Adj_Open'] <= analytics_df['Adj_Close'].shift(1)) &
                           (analytics_df['body'] < analytics_df['body'].shift(1)) &
                           (analytics_df['range'] > analytics_df['range'].shift(1)))


In [67]:
bullish_engulfing = analytics_df[ ( analytics_df['ADR_percent_5d'] > 5 ) &
                                   (analytics_df['bullish_engulfing']==True) &
                                  #(analytics_df['Adj_Vol_EMA_prev_20d'] > 150000 ) &
                                  (analytics_df['Adj_Vol'] >= 1.5 * analytics_df['Adj_Vol_EMA_prev_20d']) &
                                  (analytics_df['Adj_Close']>= 1.5) &
                                  (analytics_df['Adj_Close']<=250) &
                                  (analytics_df['Date'].isin(get_last_N_days(1)))
                                ].reset_index(drop=True)['Ticker'].values

print(bullish_engulfing)
bullish_engulfing = [x for x in bullish_engulfing]
analytics_rec_dict['bullish_engulfing'] = bullish_engulfing
len(bullish_engulfing)

['CLVT' 'LUNR' 'OUT']


3

In [68]:
analytics_df.head(2)

,company_Name,Ticker,Date,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Vol,Exchange,OHLC_price,Adj_Close_before_1d,Adj_Close_before_2d,Adj_Close_before_3d,Adj_Close_before_5d,Adj_Close_before_10d,Adj_Close_before_20d,Adj_Close_before_120d,Adj_Close_before_240d,Adj_Close_before_300d,Adj_Close_change_per_1d,Adj_Close_change_per_2d,Adj_Close_change_per_3d,Adj_Close_change_per_5d,Adj_Close_change_per_10d,Adj_Close_change_per_20d,Adj_Close_change_per_120d,Adj_Close_change_per_240d,Adj_Close_change_per_300d,Adj_Close_change_per_1d_mean10,Cam_Range_1_days,Cam_PP_day_1,Cam_R3_day_1,Cam_S3_day_1,Cam_Range_1P_days,Cam_PP_day_1P,Cam_R3_day_1P,Cam_S3_day_1P,Cam_PP_day_1_per,Cam_R3_day_1_per,Cam_S3_day_1_per,Cam_PP_day_1P_per,Cam_R3_day_1P_per,Cam_S3_day_1P_per,Cam_Range_5_days,Cam_PP_day_5,Cam_R1_day_5,Cam_S1_day_5,Cam_Range_5P_days,Cam_PP_day_5P,Cam_R1_day_5P,Cam_S1_day_5P,Cam_PP_day_5_per,Cam_R1_day_5_per,Cam_S1_day_5_per,Cam_PP_day_5P_per,Cam_R1_day_5P_per,Cam_S1_day_5P_per,Adj_High_2_days_per,Adj_Low_2_days_per,Cam_PP_day_2_per,Cam_R1_day_2_per,Cam_S1_day_2_per,Adj_High_20_days_per,Adj_Low_20_days_per,Cam_PP_day_20_per,Cam_R1_day_20_per,Cam_S1_day_20_per,Adj_High_120_days_per,Adj_Low_120_days_per,Cam_PP_day_120_per,Cam_R1_day_120_per,Cam_S1_day_120_per,Adj_High_240_days_per,Adj_Low_240_days_per,Cam_PP_day_240_per,Cam_R1_day_240_per,Cam_S1_day_240_per,RSI_9,RSI_14,RSI_20,volatility_126,max_drop_126,volatility_20,max_drop_20,TR,ATR_3,volatility_5,volatility_3,volatility_2,max_drop_5,max_drop_3,ADR_percent_5d,ADR_percent_20d,BB_2_up_20_Adj_Close,BB_2_dn_20_Adj_Close,BB_3_up_20_Adj_Close,BB_3_dn_20_Adj_Close,Adj_Close_lag1,Adj_Close_MA200,Adj_Close_MA150,Adj_Close_MA100,Adj_Close_MA90,Adj_Close_MA50,Adj_Close_MA40,Adj_Close_MA26,Adj_Close_MA20,Adj_Close_MA15,Adj_Close_MA12,Adj_Close_MA10,Adj_Close_MA5,Adj_Close_MA3,Adj_Close_MA2,D1_Close_volatility_percnt,D1_Close_volatility_percnt_before_1d,D1_Close_volatility_percnt_before_2d,D1_Close_volatility_percnt_before_3d,D1_Close_volatility_percnt_before_5d,D1_Close_volatility_percnt_before_10d,D1_Close_volatility_percnt_before_20d,price_voilatility_change_last_1d,price_voilatility_change_last_2d,price_voilatility_change_last_3d,price_voilatility_change_last_5d,price_voilatility_change_last_10d,price_voilatility_change_last_20d,gap_percent,gap_5day_max,gap_5day_min,High_52_weeks,Low_52_weeks,High_26_weeks,Low_26_weeks,High_4_weeks,Low_4_weeks,High_1_week,Low_1_week,body_percent,body_per_5d_mean,body_per_20d_mean,body_per_3month_mean,Long_MA20_crossed_MA200_in_5d,Short_MA20_crossed_MA200_in_5d,Long_MA10_crossed_MA50_in_5d,Short_MA10_crossed_MA50_in_2d,Long_MA20_crossed_MA200_in_2d,Short_MA20_crossed_MA200_in_2d,Long_MA10_crossed_MA50_in_2d,EMA_5,EMA_9,EMA_10,EMA_20,ATR,KCupper,KClower,High_Open_diff,Low_Open_diff,High_Low_diff,High_Open_diff_5d_max,High_Open_diff_5d_min,Low_Open_diff_5d_max,Low_Open_diff_5d_min,High_Low_diff_5d_max,High_Low_diff_5d_min,straight_down,straight_up,straight_down_per,straight_up_per,Close_Low_diff,Close_Open_diff,straight_up_close_per,Adj_Vol_lag1,Adj_Vol_MA60,Adj_Vol_MA20,Adj_Vol_MA10,Adj_Vol_MA5,Adj_Vol_MA2,Adj_Vol_STD20,Adj_Vol_STD10,Adj_Vol_STD5,Adj_Vol_STD2,On_Balance_Adj_Vol,Adj_Vol_before_1d,Adj_Vol_before_2d,Adj_Vol_before_3d,Adj_Vol_before_5d,Adj_Vol_before_10d,Adj_Vol_before_20d,Adj_Vol_change_per_1d,Adj_Vol_change_per_2d,Adj_Vol_change_per_3d,Adj_Vol_change_per_5d,Adj_Vol_change_per_10d,Adj_Vol_change_per_20d,VWAP_Adj_Close5,VWAP_Adj_Close3,VWAP_Adj_Close2,5_day_negatives,10_day_negatives,Adj_Vol_1d,Adj_Vol_EMA_prev_5d,Adj_Vol_EMA_prev_20d,EMA_prev_20d_DIV_Adj_Vol,EMA_prev_5d_DIV_Adj_Vol,Opening_Gap_Percent,day_Open_high_per,day_Open_low_per,day_Open_close_per,ATR_3_DIV_OHLC_price,Cam_R1_day_5P_DIV_Cam_R1_day_5,Cam_S1_day_5P_DIV_Cam_S1_day_5,Cam_R3_day_1P_DIV_Cam_R3_day_1,Cam_S3_day_1P_DIV_Cam_S3_day_1,Adj_Close_DIV_EMA_9,EMA_9_DIV_MA50,MA50_DIV_MA150,MA150_DIV_MA200,Close_DIV_High_52_weeks,Close_DIV_Low_52_weeks,Close_DIV_High_4_weeks,Close_DIV_Low_4_weeks,Clos

In [78]:
predicted_EPs = pd.read_pickle( f"{PROJ_PATH}/{PROCESSING_TEMP_FOLDER_PATH}/{[x for x in files_in_PROCESSING_TEMP if 'predicted_EPs' in x][0]}") 
predicted_EPs = [x for x in predicted_EPs['Ticker'].unique()]

In [80]:
ALL_EPs = list(set(QM_Long_tickers + QM_EP_recent1 + predicted_EPs))


All_EPs_df = analytics_df[ (analytics_df['Ticker'].isin(ALL_EPs )) &
                        (analytics_df['Date'].isin(get_last_N_days(1) )) 

                        ].reset_index(drop=True)

for price_cat in [ 'LT_2', 'GT_2_LT_5' , 'GT_5_LT_20', 'GT_20']:
    analytics_rec_dict[f'All_EPs_{price_cat}'] = [ x for x in All_EPs_df[All_EPs_df['price_cat']==price_cat]['Ticker'].unique() ]


In [81]:
analytics_rec_df = pd.DataFrame() 
analytics_rec_df['type'] = analytics_rec_dict.keys() 

analytics_rec_df['rec_tickers'] = analytics_rec_dict.values() 
analytics_rec_df['rec_tickers'] = analytics_rec_df['rec_tickers'].apply(lambda x: ','.join(x)) 
analytics_rec_df1 = analytics_rec_df.copy()

analytics_rec_df1['rec_tickers'] = analytics_rec_df1['rec_tickers'].str.replace(',', ', ')

analytics_rec_df1 


,type,rec_tickers
0,consolidating_5d,"ACHR, ADMP, AMD, DRVN, FUBO, GDRX, INVZ, RAD, SNAP, SPR, TGTX, YELL"
1,extended_fall_days,"AMRS, PTRA, UPST, WW"
2,extended_rise_days,"TDS, TRHC"
3,ramp_2_days_for_dpb_or_frd,"BNTC, TNGX"
4,FGD,"TNGX, BNTC"
5,SP_ana_top,"DHC, UPST, TDUP, HUT, CBAY, UIHC, EZGO, TRHC"
6,PP_range,"LPRO, DRUG, PALI, TWOU, RYAM, YELL, UPST, GRWG, VRM, AMRS, DOCS, SDC, WE, RBLX, NVEI"
7,long_crossover,"DRUG, FSR, NKTR, NKTR, SVFD, TNGX"
8,short_crossover,"ABCL, AMWL, AMWL, APLD, ASPN, BFLY, CYH, FSR, FTCH, IMGN, ME, RUN, TGTX"
9,bearish_engulfing,"AMWL, AVDL, BFLY, REI, RVNC, SERA"


In [82]:
analytics_df[(analytics_df['Ticker'].isin(predicted_EPs))&
             (analytics_df['Date'].isin(get_last_N_days(1)))].groupby('price_cat')['Ticker'].unique()
             

price_cat
GT_20                                                                       []
GT_2_LT_5                                   [BLUE, BNTC, EOSE, IDAI, RAD, TUP]
GT_5_LT_20                                                              [TNGX]
LT_2          [ADMP, BGLC, BIRD, COSM, CRDF, FGEN, HYZN, MF, MKFG, SVFD, YELL]
Name: Ticker, dtype: object

In [83]:
analytics_rec_df1.to_csv(f"{PROJ_PATH}/past_preds/splt_dt_{str(SPLIT_DATE).split()[0]}_result_ana.csv") 

In [84]:

focus_tickers = [] 
focus_tickers = [item.strip() for sublist in [ x.split(',') for x in analytics_rec_df1['rec_tickers']] for item in sublist if item!='']
    
focus_tickers = list(set(focus_tickers))
print(len(focus_tickers))

focus_tickers_df = pd.DataFrame(focus_tickers)
focus_tickers_df.columns = ['Ticker']
focus_tickers_df.to_csv(f"/Users/neilgsd/Downloads/ana_daily_watchlist{SPLIT_DATE}.csv", index=False) 


97


In [85]:
focus_tickers_df = pd.merge(focus_tickers_df, analytics_df, how='left', on='Ticker') 
print(focus_tickers_df.shape) 
focus_tickers_df.head(2) 

(24228, 383)


,Ticker,company_Name,Date,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Vol,Exchange,OHLC_price,Adj_Close_before_1d,Adj_Close_before_2d,Adj_Close_before_3d,Adj_Close_before_5d,Adj_Close_before_10d,Adj_Close_before_20d,Adj_Close_before_120d,Adj_Close_before_240d,Adj_Close_before_300d,Adj_Close_change_per_1d,Adj_Close_change_per_2d,Adj_Close_change_per_3d,Adj_Close_change_per_5d,Adj_Close_change_per_10d,Adj_Close_change_per_20d,Adj_Close_change_per_120d,Adj_Close_change_per_240d,Adj_Close_change_per_300d,Adj_Close_change_per_1d_mean10,Cam_Range_1_days,Cam_PP_day_1,Cam_R3_day_1,Cam_S3_day_1,Cam_Range_1P_days,Cam_PP_day_1P,Cam_R3_day_1P,Cam_S3_day_1P,Cam_PP_day_1_per,Cam_R3_day_1_per,Cam_S3_day_1_per,Cam_PP_day_1P_per,Cam_R3_day_1P_per,Cam_S3_day_1P_per,Cam_Range_5_days,Cam_PP_day_5,Cam_R1_day_5,Cam_S1_day_5,Cam_Range_5P_days,Cam_PP_day_5P,Cam_R1_day_5P,Cam_S1_day_5P,Cam_PP_day_5_per,Cam_R1_day_5_per,Cam_S1_day_5_per,Cam_PP_day_5P_per,Cam_R1_day_5P_per,Cam_S1_day_5P_per,Adj_High_2_days_per,Adj_Low_2_days_per,Cam_PP_day_2_per,Cam_R1_day_2_per,Cam_S1_day_2_per,Adj_High_20_days_per,Adj_Low_20_days_per,Cam_PP_day_20_per,Cam_R1_day_20_per,Cam_S1_day_20_per,Adj_High_120_days_per,Adj_Low_120_days_per,Cam_PP_day_120_per,Cam_R1_day_120_per,Cam_S1_day_120_per,Adj_High_240_days_per,Adj_Low_240_days_per,Cam_PP_day_240_per,Cam_R1_day_240_per,Cam_S1_day_240_per,RSI_9,RSI_14,RSI_20,volatility_126,max_drop_126,volatility_20,max_drop_20,TR,ATR_3,volatility_5,volatility_3,volatility_2,max_drop_5,max_drop_3,ADR_percent_5d,ADR_percent_20d,BB_2_up_20_Adj_Close,BB_2_dn_20_Adj_Close,BB_3_up_20_Adj_Close,BB_3_dn_20_Adj_Close,Adj_Close_lag1,Adj_Close_MA200,Adj_Close_MA150,Adj_Close_MA100,Adj_Close_MA90,Adj_Close_MA50,Adj_Close_MA40,Adj_Close_MA26,Adj_Close_MA20,Adj_Close_MA15,Adj_Close_MA12,Adj_Close_MA10,Adj_Close_MA5,Adj_Close_MA3,Adj_Close_MA2,D1_Close_volatility_percnt,D1_Close_volatility_percnt_before_1d,D1_Close_volatility_percnt_before_2d,D1_Close_volatility_percnt_before_3d,D1_Close_volatility_percnt_before_5d,D1_Close_volatility_percnt_before_10d,D1_Close_volatility_percnt_before_20d,price_voilatility_change_last_1d,price_voilatility_change_last_2d,price_voilatility_change_last_3d,price_voilatility_change_last_5d,price_voilatility_change_last_10d,price_voilatility_change_last_20d,gap_percent,gap_5day_max,gap_5day_min,High_52_weeks,Low_52_weeks,High_26_weeks,Low_26_weeks,High_4_weeks,Low_4_weeks,High_1_week,Low_1_week,body_percent,body_per_5d_mean,body_per_20d_mean,body_per_3month_mean,Long_MA20_crossed_MA200_in_5d,Short_MA20_crossed_MA200_in_5d,Long_MA10_crossed_MA50_in_5d,Short_MA10_crossed_MA50_in_2d,Long_MA20_crossed_MA200_in_2d,Short_MA20_crossed_MA200_in_2d,Long_MA10_crossed_MA50_in_2d,EMA_5,EMA_9,EMA_10,EMA_20,ATR,KCupper,KClower,High_Open_diff,Low_Open_diff,High_Low_diff,High_Open_diff_5d_max,High_Open_diff_5d_min,Low_Open_diff_5d_max,Low_Open_diff_5d_min,High_Low_diff_5d_max,High_Low_diff_5d_min,straight_down,straight_up,straight_down_per,straight_up_per,Close_Low_diff,Close_Open_diff,straight_up_close_per,Adj_Vol_lag1,Adj_Vol_MA60,Adj_Vol_MA20,Adj_Vol_MA10,Adj_Vol_MA5,Adj_Vol_MA2,Adj_Vol_STD20,Adj_Vol_STD10,Adj_Vol_STD5,Adj_Vol_STD2,On_Balance_Adj_Vol,Adj_Vol_before_1d,Adj_Vol_before_2d,Adj_Vol_before_3d,Adj_Vol_before_5d,Adj_Vol_before_10d,Adj_Vol_before_20d,Adj_Vol_change_per_1d,Adj_Vol_change_per_2d,Adj_Vol_change_per_3d,Adj_Vol_change_per_5d,Adj_Vol_change_per_10d,Adj_Vol_change_per_20d,VWAP_Adj_Close5,VWAP_Adj_Close3,VWAP_Adj_Close2,5_day_negatives,10_day_negatives,Adj_Vol_1d,Adj_Vol_EMA_prev_5d,Adj_Vol_EMA_prev_20d,EMA_prev_20d_DIV_Adj_Vol,EMA_prev_5d_DIV_Adj_Vol,Opening_Gap_Percent,day_Open_high_per,day_Open_low_per,day_Open_close_per,ATR_3_DIV_OHLC_price,Cam_R1_day_5P_DIV_Cam_R1_day_5,Cam_S1_day_5P_DIV_Cam_S1_day_5,Cam_R3_day_1P_DIV_Cam_R3_day_1,Cam_S3_day_1P_DIV_Cam_S3_day_1,Adj_Close_DIV_EMA_9,EMA_9_DIV_MA50,MA50_DIV_MA150,MA150_DIV_MA200,Close_DIV_High_52_weeks,Close_DIV_Low_52_weeks,Close_DIV_High_4_weeks,Close_DIV_Low_4_weeks,Clos

In [89]:
env_config['PORTFOLIO']['recent_imp_tickers'] = ','.join(focus_tickers)

with open(f'{PROJ_PATH}/kye/env_config.ini', 'w') as configfile:    # save
    env_config.write(configfile)